### **Introduction to Arachne**
Arachne is a Python package for graph analysis that is built as an extension to Arkouda, a Python package for analysis on tabular data, akin to NumPy and Pandas. In this notebook we will show examples on how to run each algorithm that has been implemented on different types of graphs: undirected, directed, and property.

In [1]:
import arkouda as ak
import arachne as ar
import scipy as sp
import networkx as nx
import matplotlib.pyplot as plt
import os
import time

ak.connect("n0081", 5555)
n = 100
m = 1000
s = 2
x = 2
y = 2

def read_graph_to_arkouda(file_path: str):
    """
    Reads a directed graph from a file and converts it into Arkouda arrays.

    Parameters:
        file_path (str): Path to the graph file.

    Returns:
        Tuple[ak.pdarray, ak.pdarray, int]: Tuple containing src Arkouda array, dst Arkouda array, and number of nodes.
    """
    src_list = []
    dst_list = []

    with open(file_path, 'r') as f:
        # Read the first line to get the number of nodes
        num_nodes_line = f.readline()
        try:
            num_nodes = int(num_nodes_line.strip())
        except ValueError:
            raise ValueError(f"Invalid number of nodes: '{num_nodes_line.strip()}'")

        # Read the remaining lines for edges
        for line_number, line in enumerate(f, start=2):
            # Skip empty lines
            if not line.strip():
                continue

            parts = line.strip().split('\t')
            if len(parts) != 2:
                print(f"Warning: Skipping malformed line {line_number}: '{line.strip()}'")
                continue

            try:
                src, dst = map(int, parts)
            except ValueError:
                print(f"Warning: Non-integer values on line {line_number}: '{line.strip()}'")
                continue

            src_list.append(src)
            dst_list.append(dst)

    # Convert to Arkouda arrays
    ak_src = ak.array(src_list)
    ak_dst = ak.array(dst_list)

    return ak_src, ak_dst, num_nodes

# Define the file path
file_path ="/mmfs1/home/md724/arkouda-njit/arachne/data/motifCounting/yeast"
# file_path ="/mmfs1/home/md724/arkouda-njit/arachne/data/motifCounting/social.txt"
#file_path ="/home/md724/arkouda-njit/arachne/data/motifCounting/Simple_1.txt"
#file_path = "/mmfs1/home/md724/arkouda-njit/arachne/data/motifCounting/random_0.0005_1000.txt"
#file_path = "/mmfs1/home/md724/arkouda-njit/arachne/data/motifCounting/random_0.0005_100000.txt"
#file_path = "/mmfs1/home/md724/arkouda-njit/arachne/data/motifCounting/random_0.05_1000.txt"
# file_path ="/mmfs1/home/md724/arkouda-njit/arachne/data/motifCounting/elec"

src, dst, num_nodes = read_graph_to_arkouda(file_path)

# Display the results
print(f"Number of nodes: {num_nodes}")
print(f"Number of edges: {src.size}")
### Build graph from randomly generated source and destination arrays.
# 1. Use Arkouda's randint to generate the random edge arrays.

#src = ak.array([1, 2, 1, 3, 4, 2, 5, 4, 5, 6, 3, 8, 9, 4, 9, 10])
#dst = ak.array([0, 1, 3, 4, 1, 4, 2, 5, 6, 7, 8, 9, 3, 9, 10, 4])#
# 
# src = ak.array([0, 1,2,2,3,4])
# dst = ak.array([1, 2,0,3,4,2])

# 2. Build temporary property graph to get sorted edges and nodes lists.
temp_prop_graph = ar.PropGraph()
start = time.time()
temp_prop_graph.add_edges_from(src, dst)
end = time.time()
build_time = end - start
print(f"Building property graph with {len(temp_prop_graph)} vertices and "
        f"{temp_prop_graph.size()} "
        f"edges took {round(build_time,2)} seconds.")

### Generate node labels and edge relationships for the graph.
# 1. Extract node and edge information.
num_edges = temp_prop_graph.size()
num_nodes = len(temp_prop_graph)
edges = temp_prop_graph.edges()
nodes = temp_prop_graph.nodes()

# 2. Generate sets of node labels and edge relationships.
labels_set = ak.array(["lbl" + str(x) for x in range(1,x+1)])
relationships_set = ak.array(["rel" + str(y) for y in range(1,y+1)])

# 3. Give edges and nodes same exact labels and relationships.
node_labels_1 = ak.full(num_nodes, labels_set[0])
node_labels_2 = ak.full(num_nodes, labels_set[1])
edge_rels_1 = ak.full(num_edges, relationships_set[0])
edge_rels_2 = ak.full(num_edges, relationships_set[1])

# 4. Create dataframe to load into a new property graph.
edge_df = ak.DataFrame({"src":edges[0], "dst":edges[1], "rels1":edge_rels_1, "rels2":edge_rels_2})
node_df = ak.DataFrame({"nodes":nodes, "lbls1":node_labels_1, "lbls2":node_labels_2})

# 5. Create new property graph with node labels and edge relationships.
prop_graph = ar.PropGraph()
prop_graph.load_edge_attributes(edge_df, source_column="src", destination_column="dst",
                                relationship_columns=["rels1", "rels2"])
prop_graph.load_node_attributes(node_df, node_column="nodes", label_columns=["lbls1", "lbls2"])
###########################################################################################
print("motif_counting started for ...")
start = time.time()

# isos = ar.motif_counting(prop_graph, subgraph, semantic_check="or"
isos = ar.motif_counting(prop_graph, 7 ,1,
                                             algorithm_type="si", return_isos_as="vertices")

end = time.time()
exec_time = end - start
print(f"motif_counting for ...."

        f" took {round(exec_time,3)} seconds.")
print("isos = ", isos)
# print("uniqueMotifClasses.size, globalClasses.size, globalMotifCount.read(), motifCounts.size ")
# isos = ar.motif_counting(prop_graph, 5)
ak.shutdown()



    _         _                   _       
   / \   _ __| | _____  _   _  __| | __ _ 
  / _ \ | '__| |/ / _ \| | | |/ _` |/ _` |
 / ___ \| |  |   < (_) | |_| | (_| | (_| |
/_/   \_\_|  |_|\_\___/ \__,_|\__,_|\__,_|
                                          

Client Version: v2024.06.21


connected to arkouda server tcp://*:5555


Number of nodes: 688
Number of edges: 1079
Building property graph with 688 vertices and 1079 edges took 0.21 seconds.
motif_counting started for ...
motif_counting for .... took 96.704 seconds.
isos =  (array([0]), array([0]))
